##### Importing the required libraries

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import codecs
import pdb
import time

##### Creating a variable which stores working diectory and one which stores the path where the webpagess needeth to be dumped

In [ ]:
directory=os.getcwd()
webpage_dump = directory + "\\webpages"

##### Downloading the webpages

In [ ]:
#initializing Chrome Webdriver and opening the url with the search term.
driver=webdriver.Chrome()
url="https://www.amazon.in/s?k=monitors&crid=IBS3D4BWHX6M&qid=1675433952&sprefix=monitors%2Caps%2C240&ref=sr_pg_1"
driver.get(url)
#Finding the total number of webpages 
soup=BeautifulSoup(driver.page_source, "html.parser")
page_number=soup.find_all(class_="s-pagination-strip")[0].text
Nfinder=page_number.find("N")
Page=page_number[Nfinder-2] + page_number[Nfinder-1]
Page=Page.strip(".")
Page_count=int(Page)
#running a loop with stop condition as total number of pages and then iterating through the pages and downloading them
for i in range(1,Page_count+1):
    url=f"https://www.amazon.in/s?k=monitors&page={i}&crid=IBS3D4BWHX6M&qid=1675433952&sprefix=monitors%2Caps%2C240&ref=sr_pg_{i}"
    driver.get(url)
    a=os.path.join(webpage_dump, f"{i}.html")
    b=codecs.open(a, "w", "utf-8")
    c=driver.page_source
    b.write(c)

##### Parsing the HTML and saving details to excel

In [ ]:
#initializing lists
Original_Price=[]
Renewed=[]
Discount=[]
Discount_Percentage=[]
Discounted_Price=[]
Company_Name=[]
Average_Rating=[]
Ratings_Count=[]
Model_Number=[]
Delivery_Date=[]
Screen_Size=[]
Resolution=[]
resolution_numbers=["1080", "1440", "2160", "1366", "1280", "1600"]
resolution_text=["FHD", "QHD", "UHD", "5K", "HD"]
resolution=["FHD", "2K", "4K", "5K"]
Additional_Details=[]



#finding total number of pages
with open(f"{webpage_dump}\\{1}.html", "r", encoding="UTF-8") as reader:
    html_string=reader.read()
    soup=BeautifulSoup(html_string, 'html.parser')
page_number=soup.find_all(class_="s-pagination-strip")[0].text
Nfinder=page_number.find("N")
Page=page_number[Nfinder-2] + page_number[Nfinder-1]
Page=Page.strip(".")
Page_count=int(Page)


#starting the loop with stop condition as the total number of pages
for z in range(1,Page_count+1):
    print(f"Page Number{z} Processing: \n")
    with open(f"{webpage_dump}\\{z}.html", "r", encoding="UTF-8") as reader:
        html_string=reader.read()
        soup=BeautifulSoup(html_string, 'html.parser')
    d=soup.find_all(class_="a-section a-spacing-small a-spacing-top-small")[0].text
    d=d.strip("\n")
    e=d.split("-")
    master_result=e[1].split(" ")
    f=int(master_result[0])-int(e[0]) #number of sponsored listings
    w=f 
    count=0 #number of unsponsored listings


    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        if "Sponsored" in f:
            count+=1
    w=w+count
    w=w+1



    #Computing Company Name
    for i in range(w):
        d=soup.find_all(class_="a-size-medium a-color-base a-text-normal")[i].text
        d=d.split(" ")
        if (d[0]=="Renewed"):
            Company_Name.append(d[1])
        else:
            Company_Name.append(d[0])



    #Computing Resolution
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        if resolution[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_numbers[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution_numbers[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution_numbers[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_text[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution_text[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution_text[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_text[3] in f:
            Resolution.append(resolution_text[3])
        elif resolution_numbers[3] in f:
            Resolution.append(resolution_text[4])
        elif resolution_numbers[4] in f:
            Resolution.append(resolution_text[4])
        elif resolution_numbers[5] in f:
            Resolution.append(resolution_text[4])
        else:
            Resolution.append(np.nan)




    #Computing Prices
    x=0
    y=0
    while(x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        if ("₹") in f:
            d=soup.find_all(class_="a-row a-size-base a-color-base")[y].text
            d=d.split("₹")
            
            g=d[1].replace(",", "")
            if (len(d)<4):
                Discount_Percentage.append(np.nan)
                Discount.append(np.nan)
                Discounted_Price.append(np.nan)
                Original_Price.append(float(g))
                h=float(e)
            else:
                e=d[3].replace(",", "")
                Discounted_Price.append(float(g))
                h=float(e)
                k=float(g)
                Original_Price.append(float(e)) 
                
                
                Discount.append(h-k)
                Discount_Percentage.append(int(((h-k)/h)*100))
            y+=1       
        else:
            Original_Price.append(np.nan)
            Discount.append(np.nan)
            Discount_Percentage.append(np.nan)
            Discounted_Price.append(np.nan)
        x+=1 
    




    #Computing Reviews
    x=0
    y=0
    while(x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        try:
            if "out of 5 stars" in f:
                d=soup.find_all(class_="a-row a-size-small")[y].text
                d=d.split(" ")
                Average_Rating.append(float(d[0]))
                if (len(d[5])>3):
                    e=d[5].split(",")
                    Ratings_Count.append(int(e[0]+e[1]))
                else:
                    Ratings_Count.append(int(d[5]))
                y+=1
            else:
                Average_Rating.append(np.nan)
                Ratings_Count.append(np.nan)
            x+=1
        except IndexError:
            continue
    #Computing Model Number
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        e=f.split(" ")
        Model_Number.append(e[1])


    #Computing Delivery Date
    x=0
    y=0
    months=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    month_number=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    while (x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        if 'Get' in f:
            d=soup.find_all(class_="a-color-base a-text-bold")[y].text
            d=d.split(",")
            e=d[1].split(" ")
            g=months.index(e[1])
            
            if len(e[2])==1:
                h="0"+e[2]
                k=f"2023-{month_number[g]}-{h}"  
            else:
                h=e[2]
                k=f"2023-{month_number[g]}-{h}"
            Delivery_Date.append(k)
            y+=1
        
        else:
            Delivery_Date.append(np.nan)
        x+=1
    



    #Computing all other details
    for i in range(w):
        f=soup.find_all(class_="a-size-mini a-spacing-none a-color-base s-line-clamp-2")[i].text
        Additional_Details.append(f)





    #Computing Renewed Products
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        if "(Renewed)" in f:
            Renewed.append("Yes")
        else:
            Renewed.append(np.nan)
df=pd.DataFrame({"Company_Name": Company_Name, "Model_Number": Model_Number, "Resolution": Resolution, "Original_Price": Original_Price, "Discount": Discount, "Discount_Percentage": Discount_Percentage, "Discounted_Price": Discounted_Price, "Average_Rating": Average_Rating, "Ratings_Count": Ratings_Count, "Delivery_Date": Delivery_Date, "Renewed_Staus": Renewed, "Additional_Details": Additional_Details})
df.to_excel("Amazon Monitor Scrap.xlsx", sheet_name="1")    
print("Success!")